In [1]:
cd ..

/home/mars/src_aiai


In [2]:
from src.util import gpu_setup
gpu_setup()

4 actual GPUs, 1 in use.


In [3]:
import numpy as  np
import matplotlib.pyplot as plt
from src.operators.NUFFT2D_TF import NUFFT2D_TF
from copy import deepcopy
import glob 
import tqdm
import os


# Create operators

In [10]:
Nd =(360,360)

# Kd = (720, 720)
Kd = (360, 360) # TODO look at upsampling

Jd = (6,6)
batch_size = 1

uv = np.load("./data/intermediate/TNG/NUFFT_Random_var/uv_big.npy")
uv_original = np.load("./data/intermediate/TNG/NUFFT_Random_var/uv_original.npy")
sel_original =  np.load("./data/intermediate/TNG/NUFFT_Random_var/sel.npy")

m_op = NUFFT2D_TF()
m_op.plan(uv, Nd, Kd, Jd, batch_size)

100%|██████████| 65536/65536 [00:10<00:00, 6059.01it/s]


some values lie out of the interpolation array, these are not used, check baselines


# Create measurements without noise

In [4]:
data_folder = "/share/gpu0/mars/TNG_data/processed_360"
im_paths = glob.glob(data_folder + "/*.npy")
ISNR = 30 #dB

In [9]:
x_true = []
y_clean = []
y_dirty = []

x_dirty_fixed = []
psf_fixed = []

x_dirty_varying = []
psf_varying = []

for idx, im_path in enumerate(tqdm.tqdm(im_paths[:len(im_paths)])):
    im = np.load(im_path)
    x_true.append(im)
    y_ = m_op.dir_op(im)
    
    
    data_shape = y_.shape
    sigma = np.sqrt(np.mean(np.abs(y_)**2)) * 10**(-ISNR/20)
    n = np.random.normal( 0, sigma, data_shape) + 1j * np.random.normal( 0, sigma, data_shape)
    y = y_ + n/np.sqrt(2) # correct for the complex amplitude of noise
    y = y.numpy().reshape(1,-1)
    y_clean.append(y_.numpy())
    y_dirty.append(y)
    
    x_dirty_fixed.append( m_op.adj_op_sub(y[:,sel_original], sel_original).numpy()) 
    if idx == 0:
        psf_fixed.append( m_op.adj_op_sub( np.ones_like(y[:,sel_original]), sel_original).numpy())
    else:
        psf_fixed.append( psf_fixed[0])
    
    sel = np.random.permutation(len(uv)) < len(uv)//2
    x_dirty_varying.append( m_op.adj_op_sub(y[:,sel], sel).numpy() )
    psf_varying.append(  m_op.adj_op_sub( np.ones_like(y[:,sel]), sel).numpy())



save_dir = "/share/gpu0/mars/TNG_data/preprocessed_360/"
os.makedirs(save_dir, exist_ok=True)
np.save(save_dir + "./x_true.npy", np.squeeze(np.array(x_true)))
np.save(save_dir + "./y_dirty.npy", np.squeeze(np.array(y_dirty)))
np.save(save_dir + "./y_clean.npy", np.squeeze(np.array(y_clean)))
np.save(save_dir + "./x_dirty_fixed.npy", np.squeeze(np.array(x_dirty_fixed).real))
np.save(save_dir + "./psf_fixed.npy", np.squeeze(np.array(psf_fixed)))
np.save(save_dir + "./x_dirty_varying.npy", np.squeeze(np.array(x_dirty_varying).real))
np.save(save_dir + "./psf_varying.npy", np.squeeze(np.array(psf_varying)))

100%|██████████| 13400/13400 [56:35<00:00,  3.95it/s]


# create train, test, val

In [10]:
x = np.squeeze(np.array(x_true))
y = np.squeeze(np.array(x_dirty_fixed))
uv = np.squeeze(np.array(psf_fixed))

In [16]:
p = np.random.permutation(len(x))

sel_train = p<10000
sel_val = (p>=10000) & (p<11500)
sel_test = p>=11500

x_train, y_train, uv_train = x[sel_train], y[sel_train], uv[sel_train]
x_val, y_val, uv_val = x[sel_val], y[sel_val], uv[sel_val]
x_test, y_test, uv_test = x[sel_test], y[sel_test], uv[sel_test]

data_dir = "/share/gpu0/mars/TNG_data/rcGAN/image_fixed"
np.save(data_dir + "/train/x.npy", x_train)
np.save(data_dir + "/train/y.npy", y_train)
np.save(data_dir + "/train/uv.npy", uv_train)

np.save(data_dir + "/val/x.npy", x_val)
np.save(data_dir + "/val/y.npy", y_val)
np.save(data_dir + "/val/uv.npy", uv_val)

np.save(data_dir + "/test/x.npy", x_test)
np.save(data_dir + "/test/y.npy", y_test)
np.save(data_dir + "/test/uv.npy", uv_test)

In [17]:
x = np.squeeze(np.array(x_true))
y = np.squeeze(np.array(x_dirty_varying))
uv = np.squeeze(np.array(psf_varying))

In [18]:
x_train, y_train, uv_train = x[sel_train], y[sel_train], uv[sel_train]
x_val, y_val, uv_val = x[sel_val], y[sel_val], uv[sel_val]
# Test set same as with fixed sampling
# x_test, y_test, uv_test = x[sel_test], y[sel_test], uv[sel_test]

data_dir = "/share/gpu0/mars/TNG_data/rcGAN/image_varying"
np.save(data_dir + "/train/x.npy", x_train)
np.save(data_dir + "/train/y.npy", y_train)
np.save(data_dir + "/train/uv.npy", uv_train)

np.save(data_dir + "/val/x.npy", x_val)
np.save(data_dir + "/val/y.npy", y_val)
np.save(data_dir + "/val/uv.npy", uv_val)

np.save(data_dir + "/test/x.npy", x_test)
np.save(data_dir + "/test/y.npy", y_test)
np.save(data_dir + "/test/uv.npy", uv_test)

In [19]:
x_train.shape

(10000, 360, 360)

# with PSFs

In [7]:
psfs = np.load("./notebooks/psf_256.npy")

In [8]:
psfs.shape

(1024, 256, 256)

In [25]:
dirtying = lambda im, ft_psf: np.real(m_op._kk2xx(m_op._xx2kk(im) * ft_psf))

In [32]:
x_true = []
x_dirty = []
psf_list = []


for idx, im_path in enumerate(tqdm.tqdm(im_paths[:len(im_paths)])):
    if idx % len(im_paths)//len(psfs) == 0:
        psf = psfs[idx // (len(im_paths)//len(psfs))]
        ft_psf = m_op._xx2kk(psf.astype(complex))
        
    im = np.load(im_path)[50:306, 50:306].astype(complex)

    x_true.append(im)
    dirty = dirtying(im, ft_psf)
    x_dirty.append(dirty)
    psf_list.append(psf)

100%|██████████| 13400/13400 [08:53<00:00, 25.11it/s]


In [39]:

x = np.array(x_true)
y = np.array(x_dirty)
uv = np.array(psf_list)

In [41]:
p = np.random.permutation(len(x_true))
p = np.arange(len(x_true)) # make sure there is no overlap between psfs in train and in test

sel_train = p<10000
sel_val = (p>=10000) & (p<11500)
sel_test = p>=11500

x_train, y_train, uv_train = x[sel_train], y[sel_train], uv[sel_train]
x_val, y_val, uv_val = x[sel_val], y[sel_val], uv[sel_val]
x_test, y_test, uv_test = x[sel_test], y[sel_test], uv[sel_test]

data_dir = "/share/gpu0/mars/TNG_data/rcGAN/image_psfs"
os.makedirs(data_dir, exist_ok=True)
os.makedirs(data_dir + "/train/", exist_ok=True)
os.makedirs(data_dir + "/val/", exist_ok=True)
os.makedirs(data_dir + "/test/", exist_ok=True)

np.save(data_dir + "/train/x.npy", x_train)
np.save(data_dir + "/train/y.npy", y_train)
np.save(data_dir + "/train/uv.npy", uv_train)

np.save(data_dir + "/val/x.npy", x_val)
np.save(data_dir + "/val/y.npy", y_val)
np.save(data_dir + "/val/uv.npy", uv_val)

np.save(data_dir + "/test/x.npy", x_test)
np.save(data_dir + "/test/y.npy", y_test)
np.save(data_dir + "/test/uv.npy", uv_test)